In [2]:
# -*- coding: utf-8 -*-
import os
import sys
import urllib2
import pandas as pd
import re
from bs4 import BeautifulSoup
import StringIO
import time

In [21]:
# Set saved file path
file_folder_path = '/Users/maeng-dev/Documents/GitHub/Python_Lectures/2018_02_SMU_Lecture_Slide/ch09/code/csv/'

# Set the sleep time
sleep_time_sec = 1

# Make a csv file each page  1 page have 30 question & Answer pairs
csv_iteration_num = 1 # question & Answer pairs

### if you have a problem during running this crawler use this parameters
# Loan 0 Banking 1 Creadit card 2 Insurance 3 Travel 4 Study 5 Computer 6 Developer 7 Phone_qna 8
# when you select no.2 it will be automatically begin crawling include 3,4,5,..
select_category = 3
select_start_page_no = 0 # if your last file name like this 'Loan_4~6_kor_forum.csv' please put the value as '6'

In [3]:
# Target web page
# Section Name
#  - Insurance 47,641 Questions ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=consulting&page=1&category=1&divpage=11
#  - Credit card 167,457 Questions  ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=card_consult&page=1&divpage=35
#  - Banking 10,235 Questions ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=money&page=1&category=8&search_type=sub_memo&keyword=%C0%BA%C7%E0&divpage=47
#  - Loan 757 Questions ||  http://www.ppomppu.co.kr/zboard/zboard.php?id=loan_consult&page=1&divpage=1

# http://www.ppomppu.co.kr/zboard/view.php?id=money&page=1&divpage=47&category=8&search_type=sub_memo&no=262147
# http://www.ppomppu.co.kr/zboard/view.php?id=money&amp;page=1&amp;divpage=47&amp;category=8&amp;search_type=sub_memo&amp;keyword=%EC%9D%80%ED%96%89&amp;no=262174
# 

In [8]:
def url_list_per_page(pre_url, min_page, max_page, post_url):
    url_list = []
    for i in range(min_page, max_page+1):
        url_list.append(pre_url+str(i)+post_url)
    return url_list

In [14]:
# List Page Url Patterns
# If your crawling work is interrupted by unexpected accident, you can modify the number of parameters to resume your crawling
insurance_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=consulting&page=', 1, 1591,'&category=1&divpage=11')
credit_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=card_consult&page=', 1, 5620 ,'&divpage=35' )
banking_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=money&page=', 1, 513 , '&category=8&search_type=sub_memo&keyword=은행&divpage=4')
loan_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=loan_consult&page=' , 1, 27,'&divpage=1' )
travel_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=tour&page=', 1, 2476, '&divpage=1')
study_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=study&page=',1, 1790, '&divpage=1')
com_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=computer&page=',1, 14042,'&divpage=1')
dev_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=developer&page=',1, 589 , '&divpage=1')
phone_qna_urls = url_list_per_page('http://www.ppomppu.co.kr/zboard/zboard.php?id=phone2&page=', 1, 23772, '&category=7&divpage=1')

# Contents Urls Patterns
insurance_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no='
credit_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=card_consult&page=1&divpage=35&no='
banking_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=money&page=1&divpage=47&category=8&search_type=sub_memo&no='
loan_con_url= 'http://www.ppomppu.co.kr/zboard/view.php?id=loan_consult&page=1&divpage=1&no='
travel_con_url ='http://www.ppomppu.co.kr/zboard/view.php?id=tour&page=2&divpage=1&no='
study_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=study&page=1&divpage=17&no='
com_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=computer&page=2&divpage=93&no='
dev_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=developer&page=2&divpage=4&no='
phone_qna_con_url = 'http://www.ppomppu.co.kr/zboard/view.php?id=phone2&page=2&divpage=302&category=7&no='
# for j in loan_urls:
#     print j

# parameter control
cate_p = [
            ['Loan', loan_urls, loan_con_url, 27], #0
            ['Banking',banking_urls, banking_con_url, 513], #1
            ['Credit card', credit_urls, credit_con_url, 5620], #2 
            ['Insurance',insurance_urls, insurance_con_url, 1591], #3 
            ['Travel',travel_urls, travel_con_url, 2476], #4
            ['Study',study_urls, study_con_url, 1790], #5
            ['Computer',com_urls, com_con_url, 14042], #6
            ['Dev',dev_urls, dev_con_url, 589], #7
            ['Phone_qna', phone_qna_urls, phone_qna_con_url, 23772], #8
         ]

In [15]:
### Remove special character & duplicated spaces
def special_symbol_remover(target_text):
    # exception_regex = '[%$&+]' ## Exception case: It can prononciate in Korean
    sym_regex = re.compile(ur'[^0-9a-zA-Z가-힣%$&+. ]+') ## Remove symbols without space  # symbols -> space [ ] 
    ex_decimal_point_regex = ur'[0-9][\.][0-9]+' ## Decimal point & url 
    space_regex = re.compile(ur'[ ]{2,}') ## two more spaces [ ][ ] -> one space[ ]
    dot_regex = re.compile(ur'[\.]{2,}') ## two more dots ... -> one space[ ]
    while target_text[len(target_text)-1] == '.': # whlie loop can handle multiple dots at the end of sentense 
        target_text = target_text[0:len(target_text)-1]
## Use this block for exception
#     if re.search(ex_decimal_point_regex, target_text):
#         result_text = target_text
#     elif re.search(ex_decimal_point_regex, target_text):
#         result_text = target_text
    sp_tuple = re.subn(sym_regex, r" ", target_text) # Special Characters
    dot_tuple = re.subn(dot_regex, r" ", sp_tuple[0]) # Duplicated dots remover
    space_tuple = re.subn(space_regex, r" ", dot_tuple[0]) # Duplicated Space remover
    result_text = space_tuple[0]
    return result_text 
# print special_symbol_remover('오늘 한국 증시는 30% 포인트 상승했습니다......'.decode('euc-kr').encode('utf-8'))
# print special_symbol_remover(u'오늘 금리는...    어떤가요?  ^^')
# print special_symbol_remover(u'금일 금리는 0.35p 상승하는 데 그쳤습니다...')

In [16]:
def tag_remover(target_text):
    tag_list = []
    tag_list.append(ur'<textarea class="ori_comment">')
    tag_list.append(ur'</textarea>')
    for tag in tag_list:
        target_text = re.sub(tag, '', target_text)
    return target_text

In [22]:
### parameter control 
for j in range (select_category, len(cate_p)):
    df = pd.DataFrame(columns=('Section', 'No', 'Question', 'List URL', 'Contents URL', 'Answer1', 'Answer2', 'Answer3', 'Answer4', 'Answer5'))
    category = cate_p[j][0]
    list_url_n = cate_p[j][1]
    con_url_patterns = cate_p[j][2] # To add prefix url 
    last_page_n = cate_p[j][3]
#     print category_name, url_list, '|    | ', content_url_patterns
#     category = 'banking'
    page_counter = 1 # start
    for k in list_url_n[select_start_page_no: ]:
        list_url = k
        print 'go to questions list page '+str(page_counter)
        data = urllib2.urlopen(list_url).read().decode('euc-kr', errors='ignore')
        time.sleep(sleep_time_sec) 
        soup = BeautifulSoup(data, 'html.parser')
        # CSS Selecter 
        #revolution_main_table > tbody > tr:nth-child(15) > td:nth-child(4) > a > font
        p_title = soup.select ('tr > td > a ')
        # find contents unique no
        p = re.compile("(&no=)[0-9]+")
        for i in p_title:
            for m in p.finditer(i.get('href')):
        #         print special_symbol_remover(i.text) # title
                cont_no = str(i.get('href')[m.start()+4:])
                temp_link = con_url_patterns+cont_no
                print temp_link # content link
                time.sleep(sleep_time_sec) 
                content = urllib2.urlopen(temp_link).read().decode('euc-kr', errors='ignore')
                temp_soup = BeautifulSoup(content, 'html.parser')
                selected_comments = temp_soup.select('tr > td > div')
                comments = []
                for cmt in selected_comments: 
                    temp_soup = BeautifulSoup(str(cmt), 'html.parser')
                    ## Remove repeated comments 
                    if temp_soup.textarea is None: 
                        # print temp_soup.get_text()
                        comments.append(temp_soup.get_text().replace('\r', ' ').replace('\n', ' ').replace('  ', ' '))
                    else: 
                        pass
                try:
                    # one_line_comment = special_symbol_remover(comments[0].text.splitlines()[0]) # read one line of comments
                    # one_line_comment = special_symbol_remover(comments[0].text.splitlines()[0]) # read one line of comments
                    q=0
                    temp_comments = []
                    for u in comments:
                        temp_comments.append(special_symbol_remover(u))
                        q += 1
                except IndexError: 
                    pass
                    # temp_comments[0] = '#NULL'
                ## question multiple lines
                ## body > div.wrapper > div.contents > div.container > div > table:nth-child(15) > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table > tbody > tr > td > p:nth-child(2)
                ## </p><!--"&lt;--></td></tr></table><!--/DCM_BODY--></td></tr></tbody></table>
                temp_soup2 = BeautifulSoup(content, 'html.parser')
                ## body > div.wrapper > div.contents > div.container > div > table:nth-child(15) > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table > tbody > tr > td
                selected_questions = temp_soup2.select('tbody > tr > td')
                try:
                    question = ''
                    for sq in selected_questions: 
                        # Quetion = title + content
                        question = question + i.text + sq.text
                        # Remove newline and repeated spaces
                        question = question.replace('\r', ' ').replace('\n', ' ').replace('  ', ' ')
                except: 
                    question = '#NULL'
                print question
                answer = []
                for h in range(0,5):
                    try:
                        answer.append(temp_comments[h])
                    except IndexError:
                        answer.append('#NULL')
                df = df.append( {'Section':category, 'No' : cont_no, 'Question': question, 'List URL' : list_url, 'Contents URL' : temp_link, 'Answer1' : answer[0], 'Answer2': answer[1], 'Answer3': answer[2], 'Answer4': answer[3], 'Answer5': answer[4] }, ignore_index=True)
                #commentContent_1460166
        if page_counter % csv_iteration_num == 0: 
            csv_name = file_folder_path+category+"_"+str(page_counter-csv_iteration_num+1+select_start_page_no)+"~"+str(page_counter+select_start_page_no)+"_kor_forum.csv"
            df.to_csv(csv_name, encoding='utf-8')
            print "Saved at"+csv_name
            df = pd.DataFrame(columns=('Section', 'No', 'Question', 'List URL', 'Contents URL', 'Answer1', 'Answer2', 'Answer3', 'Answer4', 'Answer5'))
        elif page_counter == last_page_n:
            csv_name = file_folder_path+category+"_"+str(page_counter-csv_iteration_num+1+select_start_page_no)+"~"+str(page_counter+select_start_page_no)+"_kor_forum.csv"
            df.to_csv(csv_name, encoding='utf-8')
            df = pd.DataFrame(columns=('Section', 'No', 'Question', 'List URL', 'Contents URL', 'Answer1', 'Answer2', 'Answer3', 'Answer4', 'Answer5'))
            print "Saved at"+csv_name
        else: 
            pass
        page_counter += 1

go to questions list page 1
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=6

http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=2509
   보험상담실 이용안내 및 인증컨설턴트 소개  원하시는 설계를 뽐뿌 컨설턴트들과 함께 만들수 있습니다.  (＊인증설계사를 통해 정확한 설명과 제대로 된 가입절차 안내를 받으실 수 있습니다.)  ">   보험상담실 이용안내 및 인증컨설턴트 소개  1 2 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74134
암환자가입가능보험  1. 20대 갑상선 유두암 2기고 현재치료중입니다 종합보험에 실비할증으로 하고싶은데 최종 치료 종료 후 얼마나 지나야 가입되나요? 2. 새로 보험 가입시 또는 가입후에 청구하게 되면 보험사에서 이전에 가입된 실비 청구내역 몇년치까지 확인하나요? 3. 암 말고 타질환 고지의무 5년인가요? 최종치료이력이 5년지나면 고지안해도 되나요?
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74133
70대 보험가입 문의합니다.  아버지가 현재 가입된 보험이 하나도 없습니다. 48년생 이시고, 당뇨가 있습니다. 실비포함 가입할수 있는 보험이 있을까요?? 유병자실비보험비용은 얼마정도일까요? * insurance 에서 게시물 이동되었습니다 (2018-11-11 17:46 / no : 72010)
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74132
암보험 문

아빠가 보험이 하나도 없어요  안녕하세요 저희 아빠 보험이 하나도 없는데요 아빠가 보험은 못믿겠다며 보험들기를 싫어하세요 듣기로는 젊은시절 지인통해 이보험 저보험 무작정 가입하시고 전부 손해보고 해지하시거나 기간이 끝나서 소멸된것 같아요  합리적인 구성을 알아본뒤에 아빠를 최대한 설득시켜보고 싶습니다 59년생, 임플란트 외에는 병력이 딱히 없으시고 건강검진도 주기적으로 받고 계신데 식단관리해서 고지혈증에 주의하라고 나왔어요 1. 실비는 회사에서 뭔가 들어준게 있음 2. 사망보험금에는 관심없음 3. 진단비 위주의 구성(암이나 주요질병..?) 4. 질병상해후유장해 80%미만 5. 일상생활배상책임 제가 원하는걸 나열해보았어요~! 더 필요한게 있다면 추천해주셔도 괜찮구요! 80세 만기를 해야할지 90세만기를 해야할지는 아직 모르겠어요 어떤 보험상품을 알아보는게 좋은지 알려주시면 감사하겠습니다 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74109
태아보험 가입 원합니다 지금 임신 초기인데 태아보험 슬슬 알아보고 가입하려구 합니다 쪽지주세요태아보험 가입 원합니다 1 2 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74108
암보험 설계 부탁드려요 47세 남자 입니다...   암보험만 집중된거 설계좀 부탁드립니다..   * insurance 에서 게시물 이동되었습니다 (2018-11-09 18:43 / no : 71987)암보험 설계 부탁드려요 1 2 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74107
65년생여자 보험 리모델링 부탁드려요  안녕하세요 65년생 여자 보험인데 전문가분들의 조언이 필요해서 글 써봅니다.  첫번째 이미지는 

27세 여 보험 리모델링으로 문의드립니다! 27세 여자로 현재 두가지 보험을 가지고 있습니다. 보험 리모델링에 관해 문의드립니다. 종신보험이 보장 받는것에 비해 보험료를 많이 내는 것이 아닌가 싶어 해지를 고민 중입니다. 해약하는것이 낫다면 어떤보험을 드는 것이 좋을지 조언 부탁드립니다! 추가로 제가 2014년에 허리디스크 진단을 받아 입원치료받은적이 있는데 보험가입시 문제가 될 수 있는지 궁금합니다.   1. 무배당 메리츠 THE 건강한 보험 1304 (월 46090원 납입)               2. 하나로 준비하는 NH 종신보험(무배당) 1604 (177140원 납입)             27세 여 보험 리모델링으로 문의드립니다! 1 2 
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74087
29세 여자 보험 문의드려요  아는 분 소개로 이것저것 가입된 것들이 있는데,  보험을 잘 모르는 상태에서 노후 대비를 하라는 아는 분들 소개로 가입한 것들입니다. (총 3개)   주변에서는 보험 드는 건 다 비용이니 지금 마이너스가 있어도 빨리 빼는 게 이득이라는 얘기를 주로 하고 있습니다.   거의 경제적으로 보험을 바라보는 편인데, 지금 해약하려 하니 빠지는 금액들이 생기네요.. 유지하는게 득이 될 지(노후자금) 아니면 빠르게 해지해야 할 지 고민이 됩니다.  많은 조언 부탁드립니다.      1. 3년 전 가입한 '한화생명 하이드림연금보험(1종)'  2. 곧 2년 되는 '메트라이프 무배당 변액연금보험 동행(1종)'           3. 1년 정도 된 '(무)수호천사디딤돌유니버셜종신(트리플체증형-15년후-표준체)' - 최저보증이율 2.9%         
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74086
32살 실비보험 문의드립니

86년생 남자 가입보험 정리하고 싶어요    현재 가입중인 상품은 KB Yes!365건강보험(16.04) 13만원->아래 약관 첨부 KB손보실손의료비보장보험(16.04) 두가지 입니다.   뇌혈관, 허혈성 진단비가 약하고 암진단 말고는 해지를 해도 될 것 같은데.. 상담 부탁드립니다.   회사원/가족력 없음(허리디스크 정도)/운전 O/86년10월생                                            [안내사항] ① 설계와 가입문의는 아래 URL의 인증컨설턴트 이미지를 클릭하여 문의 하시기 바랍니다. - 관련 URL: http://www.ppomppu.co.kr/zboard/view.php?id=consulting&no=2509 ② 빠른 설계와 가입문의를 원하시는 회원님들께서는 본게시판의 상담신청 배너를 이용 하시기 바랍니다. - 게시글의 열람은 누구나 가능하기 때문에 개인정보는 절대 기재하지 마십시요. (이름/주민번호/연락처/이메일등) ------------------------------------------<게시물 작성 시 위 내용은 삭제하고 작성하세요!>--------------------------------------------
http://www.ppomppu.co.kr/zboard/view.php?id=consulting&page=1&divpage=11&category=1&no=74081
결혼전 보험 추가가입 문의드립니다.  급여 400만원이구요   현재 보험 가입내용은   실비- 2만원 (2010년 가입 해지/리모델링 의향 없음) 비갱신형 암,심장,뇌 진단급 보험 - 10만원 (암 5천만에 2천/2천 정도로 기억. 해지/리모델링 의향 없음 20년납 90세 만기) 운전자보험 - 10만원 (적립8만원 지인한테 들어준거라;;; 추후 다이렉트로 변경할 예정입니다) 종신보험 - 13만원 (부모님께서 10년째 납입해주심 내년에 가져올 예정 20년납이라 10년 더 내면 되고 보장은 8천만원)   현재 만 28세 입니다.   내년 결혼

KeyboardInterrupt: 